IMPORT ALL PACKAGES AND MODULES 

In [1]:
import cx_Oracle
from os import pardir, makedirs, sep, walk
from os.path import abspath, exists, join
import openpyxl
from pandas import ExcelWriter
import pandas as pd
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String,  create_engine
from sqlalchemy.orm import scoped_session, sessionmaker

ModuleNotFoundError: No module named 'cx_Oracle'

In [8]:
from unupdated import mf_comm_df, mf_cons_df, cons_df, comm_df, mg_cons_df, mg_comm_df

In [9]:
from rejected import rejected_cons_df, rejected_mfcons_df, rejected_mgcons_df, rejected_comm_df, rejected_mfcomm_df,rejected_mgcomm_df

DICTIONARY DEFINITION

In [10]:
stale_dict = {
    'Consumer':{
        'CMB':'cons_df',
        'MFI':'mf_cons_df',
        'PMI':'mg_cons_df'
    },
    'Corporate':{
        'CMB':'comm_df',
        'MFI':'mf_comm_df',
        'PMI':'mg_comm_df'
    }    
}

In [11]:
rej_dict = {
    'Consumer':{
        'CMB':'rejected_cons_df',
        'MFI':'rejected_mfcons_df',
        'PMI':'rejected_mgcons_df',
    },
    'Corporate':{
        'CMB':'rejected_comm_df', 
        'MFI':'rejected_mfcomm_df', 
        'PMI':'rejected_mgcomm_df', 
    }    
}

ESTABLISH CONNECTION TO THE DATABASE SYSTEM

In [12]:
pub = '(DESCRIPTION = (ADDRESS = (PROTOCOL = TCP)(HOST = 172.16.1.18)(PORT = 1521))'
pub += '(CONNECT_DATA = (SERVER = DEDICATED) (SERVICE_NAME = sb2prod) ) )'
pub_con = create_engine("oracle+cx_oracle://sbapi:sbapi@" + pub)

In [13]:
usr_dir = abspath(join(abspath('__file__'), pardir))
usr_dir

'C:\\Users\\CRCOPS002\\Documents\\Un-updated extraction'

In [14]:
pd.read_sql("select * from dual", pub_con)

,dummy
0,X


IMPORT THE SOURCE TABLE FOR PROCESSED INSTITUTIONS

In [15]:
my_df = pd.read_excel("source_data.xlsx", sep='|')
my_df.head()

,CATEGORY,NAME,DPID,SECTOR
0,Micro Lenders,AJEWOLE MICROFINANCE BANK LIMITED,2ABL100711,MFI
1,Micro Lenders,EGWAFIN MICROFINANCE BANK LIMITED,2ABL100550,MFI
2,Micro Lenders,GLORY MICROFINANCE BANK LIMITED,2ABL100812,MFI
3,Micro Lenders,NEW GOLDEN PASTURES MICROFINANCE BANK LIMITED,2ABL100208,MFI
4,Micro Lenders,ADA MICROFINANCE BANK LIMITED,2ABL100371,MFI


FILE DIRECTORY/PATH DEFINITION

In [16]:
def file_dir(cur_data_cy, name):
    pdir = abspath(join(abspath('__file__'), pardir))
    dir2u = abspath(join(pdir, str(cur_data_cy)))
    dir2u = abspath(join(dir2u, name))
    print(dir2u)
    
    if not exists(dir2u): makedirs(dir2u)
    
    return dir2u

In [17]:
# my_df.columns

In [18]:
{x.DPID: (x.NAME, x.SECTOR ) for x in my_df.sample(5).itertuples()}

{'2ABL100052': ('FIDFUND MICROFINANCE BANK', 'MFI'),
 '2ABL101106': ('FIRST ACCESS MICROFINANCE BANK LIMITED', 'MFI'),
 '2A3L100312': ('CUSTODIAN AND ALLIED INSURANCE LIMITED', 'CMB'),
 '2ABL100784': ('MAGAJINGARI MICROFINANCE BANK LIMITED', 'MFI'),
 '2A2L100336': ('FREBSON INVESTMENT LIMITED', 'CMB')}

UN-UPDATED RECORDS EXTRACTION

In [1]:
for dp in my_df.DPID.unique():
    writer = None
    for dpid, dtls in {x.DPID: (x.NAME, x.SECTOR) for x in my_df[my_df.DPID==dp].itertuples()}.items():
        name, sector = dtls
        for ctype in stale_dict:
            try:
                code = f"{stale_dict[ctype][sector]}('{dpid}')"
                print('\n', code, '\n')
                df = eval(code)
                if not df.empty:
                    cur_data_cy = df.CURRENT_DATA_CYCLE.unique()[0]
                    dir2u2sf = file_dir(cur_data_cy, name)
                    xlfn = f"Un-Updated Accounts as at {cur_data_cy} for {name}.xlsx"
                    print(xlfn)
                    writer = pd.ExcelWriter(f"""{dir2u2sf}{sep}{xlfn}""", engine='xlsxwriter') if not writer else writer

                    df.drop(columns=[
                            'CURRENT_DATA_CYCLE', 'INSTITUTION_NAME', 'DATE_EXTRACTED', 'TYPE'
                            ], inplace=True)                

                    df.to_excel(writer, index=False, sheet_name=ctype)
                print(dpid, name, ctype, df.shape)
                
            except Exception as e:
                print(e)
    writer.save() if writer else None
    writer = None
    print('#'*100)
        
    

NameError: name 'my_df' is not defined

REJECTED RECORDS EXTRACTION

In [20]:
for dp in my_df.DPID.unique():
    writer = None
    for dpid, dtls in {x.DPID: (x.NAME, x.SECTOR) for x in my_df[my_df.DPID==dp].itertuples()}.items():
        name, sector = dtls
        for ctype in rej_dict:
            try:
                code = f"{rej_dict[ctype][sector]}('{dpid}')"
                print('\n', code, '\n')
                df = eval(code)
                if not df.empty:
#                     cur_data_cy = df.CURRENT_DATA_CYCLE.unique()[0]
                    dir2u2sf = file_dir(cur_data_cy, name)
                    xlfn = f"Rejected Accounts as @ {cur_data_cy} for {name}.xlsx"
                    print(xlfn)
                    writer = pd.ExcelWriter(f"""{dir2u2sf}{sep}{xlfn}""", engine='xlsxwriter') if not writer else writer
                    #df.drop(columns=['CURRENT_DATA_CYCLE'], inplace=True)
                
                    df.to_excel(writer, index=False, sheet_name=ctype)
                print(dpid, name, ctype, df.shape)
                
            except Exception as e:
                print(e)
    writer.save() if writer else None
    writer = None
    print('#'*100)
        
    


 rejected_mfcons_df('2ABL100711') 

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\AJEWOLE MICROFINANCE BANK LIMITED
Rejected Accounts as @ 202011 for AJEWOLE MICROFINANCE BANK LIMITED.xlsx
2ABL100711 AJEWOLE MICROFINANCE BANK LIMITED Consumer (268, 2)

 rejected_mfcomm_df('2ABL100711') 

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\AJEWOLE MICROFINANCE BANK LIMITED
Rejected Accounts as @ 202011 for AJEWOLE MICROFINANCE BANK LIMITED.xlsx
2ABL100711 AJEWOLE MICROFINANCE BANK LIMITED Corporate (4, 2)
####################################################################################################

 rejected_mfcons_df('2ABL100550') 

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\EGWAFIN MICROFINANCE BANK LIMITED
Rejected Accounts as @ 202011 for EGWAFIN MICROFINANCE BANK LIMITED.xlsx
2ABL100550 EGWAFIN MICROFINANCE BANK LIMITED Consumer (30, 2)

 rejected_mfcomm_df('2ABL100550') 

2ABL100550 EGWAFIN MICROFINANCE BANK LIMITED Corporate (0, 2)
#######

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\CHIBUEZE MICROFINANCE BANK LIMITED
Rejected Accounts as @ 202011 for CHIBUEZE MICROFINANCE BANK LIMITED.xlsx
2ABL100685 CHIBUEZE MICROFINANCE BANK LIMITED Corporate (8, 2)
####################################################################################################

 rejected_mfcons_df('2ABL100649') 

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\EASY ACCESS MICROFINANCE BANK LIMITED
Rejected Accounts as @ 202011 for EASY ACCESS MICROFINANCE BANK LIMITED.xlsx
2ABL100649 EASY ACCESS MICROFINANCE BANK LIMITED Consumer (32, 2)

 rejected_mfcomm_df('2ABL100649') 

2ABL100649 EASY ACCESS MICROFINANCE BANK LIMITED Corporate (0, 2)
####################################################################################################

 rejected_mfcons_df('2ABL100761') 

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\RIMA MICROFINANCE BANK LIMITED
Rejected Accounts as @ 202011 for RIMA MICROFINANCE BANK LIMITE

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\NKPOLOGWU MICROFINANCE BANK LIMITED
Rejected Accounts as @ 202011 for NKPOLOGWU MICROFINANCE BANK LIMITED.xlsx
2ABL100388 NKPOLOGWU MICROFINANCE BANK LIMITED Consumer (7, 2)

 rejected_mfcomm_df('2ABL100388') 

2ABL100388 NKPOLOGWU MICROFINANCE BANK LIMITED Corporate (0, 2)
####################################################################################################

 rejected_mfcons_df('2ABL101017') 

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\IMSU MICROFINANCE BANK LIMITED
Rejected Accounts as @ 202011 for IMSU MICROFINANCE BANK LIMITED.xlsx
2ABL101017 IMSU MICROFINANCE BANK LIMITED Consumer (55, 2)

 rejected_mfcomm_df('2ABL101017') 

2ABL101017 IMSU MICROFINANCE BANK LIMITED Corporate (0, 2)
####################################################################################################

 rejected_mgcons_df('2A4L290067') 

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\REFUGE MORTGAGE BAN

2ABL100658 AWKA MICROFINANCE BANK Corporate (0, 2)
####################################################################################################

 rejected_cons_df('2A2L101010') 

(cx_Oracle.DatabaseError) ORA-00923: FROM keyword not found where expected
[SQL: with cys as (
  select
    to_char(add_months(sysdate, -1), 'yyyymm') current_data_cycle
    to_char(sysdate, 'yyyymm') current_data_position
  from dual
)
    select
        credit_facility_account_number account_number, error_description
    from sbhistory.SB_CONSOLIDATE_ERROR_LOG
    where file_id IN (
        select
            i.file_id
        from ioc_sb_file_info i, cys c, sb_institution_master sim
        where to_char(i.date_reported,'YYYYMM') = current_data_cycle
        and i.institution_id = sim.institution_sequence
        --and i.file_name not like '%-0000%'
        and i.file_name like '%-CON-%'
        and dpid = '2A2L101010'
    )
    AND SEVERITY LIKE '%S%'
    ]
(Background on this error at: http://sqla

 rejected_cons_df('2AFL100265') 

(cx_Oracle.DatabaseError) ORA-00923: FROM keyword not found where expected
[SQL: with cys as (
  select
    to_char(add_months(sysdate, -1), 'yyyymm') current_data_cycle
    to_char(sysdate, 'yyyymm') current_data_position
  from dual
)
    select
        credit_facility_account_number account_number, error_description
    from sbhistory.SB_CONSOLIDATE_ERROR_LOG
    where file_id IN (
        select
            i.file_id
        from ioc_sb_file_info i, cys c, sb_institution_master sim
        where to_char(i.date_reported,'YYYYMM') = current_data_cycle
        and i.institution_id = sim.institution_sequence
        --and i.file_name not like '%-0000%'
        and i.file_name like '%-CON-%'
        and dpid = '2AFL100265'
    )
    AND SEVERITY LIKE '%S%'
    ]
(Background on this error at: http://sqlalche.me/e/4xp6)

 rejected_comm_df('2AFL100265') 

2AFL100265 MANTRAC NIGERIA LIMITED Corporate (0, 2)
##################################################

...

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\ZEFA MICROFINANCE BANK LIMITED
Rejected Accounts as @ 202011 for ZEFA MICROFINANCE BANK LIMITED.xlsx
2ABL100986 ZEFA MICROFINANCE BANK LIMITED Consumer (51, 2)

 rejected_mfcomm_df('2ABL100986') 

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\ZEFA MICROFINANCE BANK LIMITED
Rejected Accounts as @ 202011 for ZEFA MICROFINANCE BANK LIMITED.xlsx
2ABL100986 ZEFA MICROFINANCE BANK LIMITED Corporate (4, 2)
####################################################################################################

 rejected_cons_df('2A2L100078') 

(cx_Oracle.DatabaseError) ORA-00923: FROM keyword not found where expected
[SQL: with cys as (
  select
    to_char(add_months(sysdate, -1), 'yyyymm') current_data_cycle
    to_char(sysdate, 'yyyymm') current_data_position
  from dual
)
    select
        credit_facility_account_number account_number, error_description
    from sbhistory.SB_CONSOLIDATE_ERROR_LOG
    where file_id IN (
   

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\QUICKFUND MICROFINANCE BANK LIMITED
Rejected Accounts as @ 202011 for QUICKFUND MICROFINANCE BANK LIMITED.xlsx
2ABL101035 QUICKFUND MICROFINANCE BANK LIMITED Consumer (83, 2)

 rejected_mfcomm_df('2ABL101035') 

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\QUICKFUND MICROFINANCE BANK LIMITED
Rejected Accounts as @ 202011 for QUICKFUND MICROFINANCE BANK LIMITED.xlsx
2ABL101035 QUICKFUND MICROFINANCE BANK LIMITED Corporate (4, 2)
####################################################################################################

 rejected_mfcons_df('2ABL100600') 

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\IHIALA MICROFINANCE BANK
Rejected Accounts as @ 202011 for IHIALA MICROFINANCE BANK.xlsx
2ABL100600 IHIALA MICROFINANCE BANK Consumer (60, 2)

 rejected_mfcomm_df('2ABL100600') 

2ABL100600 IHIALA MICROFINANCE BANK Corporate (0, 2)
#####################################################################

2ABL101004 GOLDENCHOICE MICROFINANCE BANK LIMITED Corporate (0, 2)
####################################################################################################

 rejected_mfcons_df('2ABL100443') 

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\BAINES CREDIT MICROFINANCE BANK LIMITED
Rejected Accounts as @ 202011 for BAINES CREDIT MICROFINANCE BANK LIMITED.xlsx
2ABL100443 BAINES CREDIT MICROFINANCE BANK LIMITED Consumer (64, 2)

 rejected_mfcomm_df('2ABL100443') 

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\BAINES CREDIT MICROFINANCE BANK LIMITED
Rejected Accounts as @ 202011 for BAINES CREDIT MICROFINANCE BANK LIMITED.xlsx
2ABL100443 BAINES CREDIT MICROFINANCE BANK LIMITED Corporate (21, 2)
####################################################################################################

 rejected_cons_df('2AFL100255') 

(cx_Oracle.DatabaseError) ORA-00923: FROM keyword not found where expected
[SQL: with cys as (
  select
    to_char(add_months(sysd

2A2L100139 MICRO LEASING LIMITED Corporate (0, 2)
####################################################################################################

 rejected_mfcons_df('2ABL101046') 

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\BISHOPGATE MICROFINANCE BANK LIMITED
Rejected Accounts as @ 202011 for BISHOPGATE MICROFINANCE BANK LIMITED.xlsx
2ABL101046 BISHOPGATE MICROFINANCE BANK LIMITED Consumer (468, 2)

 rejected_mfcomm_df('2ABL101046') 

2ABL101046 BISHOPGATE MICROFINANCE BANK LIMITED Corporate (0, 2)
####################################################################################################

 rejected_cons_df('2A2L100921') 

(cx_Oracle.DatabaseError) ORA-00923: FROM keyword not found where expected
[SQL: with cys as (
  select
    to_char(add_months(sysdate, -1), 'yyyymm') current_data_cycle
    to_char(sysdate, 'yyyymm') current_data_position
  from dual
)
    select
        credit_facility_account_number account_number, error_description
    from sbhisto

2A2L101077 CORDROS CAPITAL LIMITED Corporate (0, 2)
####################################################################################################

 rejected_mfcons_df('2ABL101128') 

C:\Users\CRCOPS002\Documents\Un-updated extraction\202011\BIDA POLYTECHNIC MICROFINANCE BANK LIMITED
Rejected Accounts as @ 202011 for BIDA POLYTECHNIC MICROFINANCE BANK LIMITED.xlsx
2ABL101128 BIDA POLYTECHNIC MICROFINANCE BANK LIMITED Consumer (12, 2)

 rejected_mfcomm_df('2ABL101128') 

2ABL101128 BIDA POLYTECHNIC MICROFINANCE BANK LIMITED Corporate (0, 2)
####################################################################################################

 rejected_cons_df('2AFL101266') 

(cx_Oracle.DatabaseError) ORA-00923: FROM keyword not found where expected
[SQL: with cys as (
  select
    to_char(add_months(sysdate, -1), 'yyyymm') current_data_cycle
    to_char(sysdate, 'yyyymm') current_data_position
  from dual
)
    select
        credit_facility_account_number account_number, error_des